# Iterative Backtesting - Part 4 (Close Positions)

In [ ]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

In [ ]:
class Backtester():
    def __init__(self, ticker, ema_short_period, ema_long_period, amount):
        self.symbol = ticker
        self.ema_s = ema_short_period
        self.ema_l = ema_long_period
        self.result = None
        self.trades = 0
        self.units = 0
        self.logs = pd.DataFrame()
        self.data = pd.DataFrame()
        self.position = 0
        self.initial_balance = amount
        self.current_balance = amount

        
    def fetch_data(self):
        # API, Database, csv file
        data = pd.read_csv('eurusd.csv')
        data['date'] = pd.to_datetime(data['date'])
        data.set_index('date', drop=True, inplace=True)
        data['return'] = np.log(data.close/data.close.shift(1))
        data['ema_short'] = data['close'].ewm(span=self.ema_s, adjust=False).mean()
        data['ema_long'] = data['close'].ewm(span=self.ema_l, adjust=False).mean()
        self.data = data

    def access_data(self, bar):
        date_value = self.data.index[bar].date()
        price = float(self.data['close'].iloc[bar])
        ema_short = float(self.data['ema_short'].iloc[bar])
        ema_long = float(self.data['ema_long'].iloc[bar])
        return date_value, price, ema_short, ema_long

    def get_balance(self):
        return self.current_balance
        
    def get_current_position(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        print(f" Current Position Value: {current_position}")

    def get_current_NAV(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        current_NAV = self.current_balance + current_position
        print(f" Current NAV: {current_NAV}")
        
    def buy_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units + qty
        self.current_balance = self.current_balance - (price * qty)
        self.trades = self.trades + 1
        print(f"{date_value} | BUY {qty} @ $: {round(price, 2) }")
    def sell_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units - qty
        self.current_balance = self.current_balance + (price * qty)
        self.trades = self.trades + 1        
        print(f"{date_value} | SELL {qty} @ $: {round(price, 2) }")
        
    def close_positions(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        if self.position != 0:
            print("\n")
            print('$$$ - Closing positions Finally - $$$')
            if self.position == 1:
                self.sell_instrument(bar = bar, qty = abs(self.units))
            elif self.position == -1:
                self.buy_instrument(bar = bar, qty = abs(self.units))
            self.position = 0
        performance = ((self.current_balance - self.initial_balance) / self.initial_balance) * 100
        print("\n")
        print("------------------- Summary -----------------")
        print(f"Performance: {round(performance, 2)} %")
        self.net_balance = self.get_balance()
        print(f"Current Balance: {self.net_balance}")
        print(f"Number of Trades: {self.trades}")


In [ ]:
obj = Backtester(ticker = 'NIFTY', ema_short_period = 50, ema_long_period = 200, amount = 70000)
obj.fetch_data()

## Iterate Over Data

In [ ]:
for i in range(10):
    print(i)

In [ ]:
n = 10  # number of rows you want
for date, row in obj.data.head(n).iterrows():
    print(date, row['close'], row['ema_short'], row['ema_long'])


In [ ]:
n = 10  # number of rows
for i in range(n):
    row = obj.data.iloc[i]
    date = obj.data.index[i]
    print(date, row['close'], row['ema_short'], row['ema_long'])
